In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import time

custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.Flowers102(
    root="data",
    split= "train",
    download=True,
    transform= custom_transform,
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split= "val",
    download=True,
    transform= custom_transform,
)

val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

count=0
for i, (X_Train, y_train) in enumerate(training_data):
  count+= 1
print(X_Train.shape)
conv1 = nn.Conv2d(3, 32, 3, 1, 1)
conv2 = nn.Conv2d(32, 64, 3, 1, 1)
conv3 = nn.Conv2d(64, 128, 3, 1, 1)
n = F.relu(conv1(X_Train))

n = F.max_pool2d(n, 2, 2)


n = F.relu(conv2(n))

n = F.max_pool2d(n, 2, 2)
n = F.relu(conv3(n))

n = F.max_pool2d(n, 2, 2)
print(n.shape)

Using cuda device
torch.Size([3, 224, 224])
torch.Size([128, 28, 28])


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 102)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        #First Pass
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        #Second Pass
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        #Third Pass
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 128 * 28 * 28)
        #Fully Connected Layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:

model  =  NeuralNetwork()
model


NeuralNetwork(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=102, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    return val_loss, correct*100

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
# Training
start = time.time()
epochs = 100
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn)
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")
    scheduler.step(val_loss)  # Adjust learning rate based on validation loss
end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/100
Train Loss: 4.7844, Val Loss: 4.6239, Val Accuracy: 0.78
Epoch 2/100
Train Loss: 4.6209, Val Loss: 4.5999, Val Accuracy: 2.06
Epoch 3/100
Train Loss: 4.5605, Val Loss: 4.4312, Val Accuracy: 2.16
Epoch 4/100
Train Loss: 4.4275, Val Loss: 4.2420, Val Accuracy: 3.82
Epoch 5/100
Train Loss: 4.3048, Val Loss: 4.1773, Val Accuracy: 3.43
Epoch 6/100
Train Loss: 4.1684, Val Loss: 4.0758, Val Accuracy: 3.73
Epoch 7/100
Train Loss: 4.1655, Val Loss: 4.0357, Val Accuracy: 5.10
Epoch 8/100
Train Loss: 4.0377, Val Loss: 3.9697, Val Accuracy: 6.67
Epoch 9/100
Train Loss: 4.0005, Val Loss: 3.8680, Val Accuracy: 7.35
Epoch 10/100
Train Loss: 3.9468, Val Loss: 3.8578, Val Accuracy: 9.51
Epoch 11/100
Train Loss: 3.8922, Val Loss: 3.8422, Val Accuracy: 7.94
Epoch 12/100
Train Loss: 3.8740, Val Loss: 3.7842, Val Accuracy: 8.63
Epoch 13/100
Train Loss: 3.7594, Val Loss: 3.7461, Val Accuracy: 9.61
Epoch 14/100
Train Loss: 3.6990, Val Loss: 3.7314, Val Accuracy: 11.18
Epoch 15/100
Train Loss: 3.6

KeyboardInterrupt: 